# SWEMLv2.0 Hold Water Years out script
This design of the script is to align with the SWE test bed set up by Andy Wood and Ethan Ritchie. The testbed uses ASO scenes from WYs 2019-2021 to evaluate SWE model estimate predictive performance within the NextGen HydroFabric framework. In addition to the SWE testbed, this script supportd direct comparison of x M resolution model predictions with the respective ASO image (e.g., not within the hydrofabric)

In [ ]:
# basic packages
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pickle

#Shared/Utility scripts
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from model_scripts import Simple_Eval, dataloader, dataprocessor2, xgb_model, SSWEET, DF_to_GeoTiff
HOME = os.getcwd()

modelname = 'XGBoost'
model_path = f"{HOME}/SWEMLv2.0/Model/{modelname}"
if not os.path.exists(model_path):
    os.makedirs(model_path, exist_ok=True)
print(f"{modelname} development script, {device}")

In [ ]:
  input_columns = [
                        'cen_lat',	
                        'cen_lon',	
                        'Elevation_m',	
                        'Slope_Deg',	
                        # 'Aspect_Deg',	
                        'Aspect_N',
                        'Aspect_W',
                        'ns_1',	
                        'ns_2',	
                        'ns_3',	
                        'ns_4',	
                        'ns_5',	
                        'ns_6',	
                        'VIIRS_SCA', 
                        'hasSnow',
                        # 'Daymet',
                        'NLDAS',
                        # 'PRISM',
                        # 'gridMET',
                        # 'AORC',
                        # 'season_precip_cm',
                     #   'region_class',
                        'DOS', 
                        'WY_week',
                        'ns_1_week_mean', 
                        'ns_2_week_mean', 
                        'ns_3_week_mean', 
                        'ns_4_week_mean',
                        'ns_5_week_mean', 
                        'ns_6_week_mean', 
                        'ns_1_anomoly',
                        'ns_2_anomoly',
                        'ns_3_anomoly', 
                        'ns_4_anomoly',
                        'ns_5_anomoly', 
                        'ns_6_anomoly',
                        'sturm_value',
                    # 'vegetation_value'
            ]
    
file_path = f"{HOME}/SWEMLv2.0/data/ASO/date_basin.pkl" 


# Train the model on all years but the testing year of interest

In [ ]:
'''
Testing for this model is on multiple scenes from WY 2019-2021. 
Model training includes all other ASO imagery.
'''
#Model set up
DataFrame = 'AORCgridMETNLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
fSCA_thresh = '10_fSCA_Thresh' # from the literature, I think this needs to be relaxed to 5 or 10%
remove0swe = True #set to False to use all data, set to True to remove values under a certain threshold
removeswe_thresh = 5 #value in cm to remove ASO obs/rows from. most papers use 10 cm...
# Years = ['2013','2014','2015','2016','2017','2018','2019']
Years =   [
    '2013','2014','2015', '2016', 
    '2017', '2018', '2019', '2020', '2021','2022', '2023'
] 
Train = True #set to true if you are training a model, False if you just want to make predictions or adjust marker size of images, etc.
DataUsed = [False] #use all data for training (True) and test with HOO, or have training/testing and HOO (False) - False leading to better model
SampleMethods =  [False] #Randomly sample all data before splitting for train/test (TRue), split train/test first, then randomize training (False)  - False leading to better model
Res = ['750M_Resolution'] #'300M_Resolution', '500M_Resolution', '750M_Resolution', '1000M_Resolution'
tries = 1 
hyperparameters = {
    'max_depth': [3,4,6,8,10],     #[8], #range (7, 9,1), #7-good
    'n_estimators': [500,1000,3500,5000,10000],#  [20,500,3500],#[3500], #range(3000, 3600, 500),  #5500 - at max tested estimator, test higher and lower values on HOO data
    'eta': [0.1],
    'objective': ['reg:squarederror'], #try the 'reg:pseudohubererror', huber_slope=1.0, or 'reg:absoluteerror' or 'reg:squaredlogerror'
     #'objective': ['reg:squarederror'], 
    # 'huber_slope':.02,
     #'max_leaves':[32], #change max_depth to 0 when using this
     #'grow_policy':['lossguide'],
     #'min_child_weight': [1],
       'random_state' : [42],
       'n_jobs' : [-1]
}
gridsearch = True #False is good for quick analysis, set to True to optimize the model, True doesnt seem to work for the pseudohubererror
perc_data = 0.30 # percent of training data used to identify optimal hyperparameters
Use_fSCA_Threshold = True

#set figure params
markersize = 50

#train each model and predict
for output_res in Res:
    for dataused in DataUsed:
        for standard in SampleMethods:
            for year in Years:
                model_path = f"{HOME}/SWEMLv2.0/Model/{modelname}/{DataFrame}/{year}"
                if not os.path.exists(model_path):
                    os.makedirs(model_path, exist_ok=True)

                sim  =  f"Test_WY{year}" 
                #load data
                WY_list = ['2013', '2014', '2015', '2016', '2017', '2018', '2019' , '2020', '2021','2022', '2023'
                ]
                WY_list.remove(year)
                alldata = dataloader.get_ML_Data(WY_list, output_res, DataFrame, fSCA_thresh)

                #small adjustments to correct some data
                #aspect
                # alldata['Aspect_Deg'] = (-alldata['Aspect_Deg']+270)%360
                # df = alldata[alldata['Aspect_Deg']>= 0]
                # df['Aspect_Deg'] = abs(df['Aspect_Deg']-90)+270
                # alldata.update(df)
                # alldata['Aspect_Deg'] = abs(alldata['Aspect_Deg'])

                '''for Vegetation_Sturm_Seasonality_PrecipVIIRSGeoObsDFs, ASO obs
                put SW 2018-04-22, 2018-03-04, San Joaquin, in bad data folder seems too big for peaks
                in Southern Rockies 2019-04-07, East River, way too high of values, put in bad data folder'''

                #clean the data
                df = alldata.copy()
                df = dataprocessor2.data_clean(df, WY_list)
                df.head()

                #temporary for seasonality relationship
                df.fillna(1, inplace = True)
                # temporary for daymet dfs
                df.reset_index(inplace=True)
                df.drop(columns='index',inplace=True)

                #convert dates to datetime format
                df.Date = pd.to_datetime(df.Date)

                years = False
                splitratio = 0.1
                test_years = [2019] #this currently does not matter
                target = 'swe_cm'
                #remove the large amounts of zero values

                df = dataloader.remove0swe(df, remove0swe, removeswe_thresh)
                
                if Train == True:

                  #fit a scaler,save, and scale the training data
                    x_train, y_train, x_test, y_test = dataprocessor2.xgb_processor(
                                                                        WY_list,
                                                                        df, 
                                                                        years, 
                                                                        splitratio,
                                                                        test_years, 
                                                                        target, 
                                                                        input_columns, 
                                                                        model_path, 
                                                                        scalertype = 'Standard',
                                                                        standard = standard,
                                                                        alldata = dataused #setting this to true will use all available data to train the model, uses HOO to test, There will not be any plots
                                                                        )

                    #Train model
                    bestparams = xgb_model.XGB_Train(model_path, 
                                        input_columns, 
                                        x_train, 
                                        y_train, 
                                        tries, 
                                        hyperparameters,
                                        perc_data,
                                        gridsearch)


                print(f"Making predictions on held out WY{year}, at {output_res}M resolution.")
                #load performance dataframes
                try:
                    print('Loading previous performance DataFrames')
                    PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}_{output_res}.parquet")
                    VDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/VDF_{sim}_{output_res}.parquet")
                    SWEDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/SWEDF_{sim}_{output_res}.parquet")

                except:    
                    print('No performance DataFrames found, making them...')
                    PDF = pd.DataFrame()
                    VDF = pd.DataFrame()
                    SWEDF = pd.DataFrame()


                #make a predition, save prediction, evaluate scene, add to overal model performance list, make cdf
                directory_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{year}/{output_res}/{DataFrame}/{fSCA_thresh}"
                #get all processed filesnames
                files = [filename for filename in os.listdir(directory_path) if filename.endswith('.parquet')]
               # remfile = '.ipynb_checkpoints'
                # for entry in os.listdir(directory_path):
                #     files.append(entry)
                #files.remove(remfile)
                #make predictions on each date
                for file in files:
                    #Get watershed name
                    watershed = file.split('_')[-2]
                    
                    #read file and split into x_test, y_test
                    TestArea = pd.read_parquet(f"{directory_path}/{file}")
                    TestArea.reset_index(inplace=True)
                    testdate = TestArea['Date'][0]
                    testdate = testdate.strftime("%Y-%m-%d")
                    #print watershed and date for easy reference
                    print(f"Making prediction on the {watershed} watershed for {testdate}")
                    
                    
                    TestArea['swe_cm'] = TestArea['swe_m'] *100
                    #remove outliers
                    TestArea = TestArea[TestArea['swe_cm']<300]

                    #Prep prediction data
                    y_test_Area = pd.DataFrame(TestArea['swe_cm'])
                    dropcols = ['cell_id','Date', 'swe_cm', 'swe_m']
                    x_test_Area = TestArea.drop(columns=dropcols)
                    x_test_Area = x_test_Area[input_columns]

                     #make a prediction
                    holdoutdate = xgb_model.XGB_Predict(
                                        model_path, 
                                        modelname, 
                                        x_test_Area,
                                        y_test_Area,
                                        Use_fSCA_Threshold
                                        )
                    #Add geospatial information to prediction DF
                    EvalDF = pd.concat([TestArea, holdoutdate], axis=1)
                    EvalDF = EvalDF.loc[:,~EvalDF.columns.duplicated()].copy()

                    # EvalDF.head(5)

                    Ppath = f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/{output_res}/{fSCA_thresh}/{year}"
                    if not os.path.exists(Ppath):
                        os.makedirs(Ppath, exist_ok=True)

                    #save the model predictions
                    table = pa.Table.from_pandas(EvalDF)
                    # Parquet with Brotli compression
                    pq.write_table(table, f"{Ppath}/HoldWYsout_{output_res}_{watershed}_{testdate}.parquet", compression='BROTLI')

                    
                    
                    #Begin Evaluation
                    savfig = True
                    markersize = 75
                    figlocation = f"{HOME}/SWEMLv2.0/Evaluation/Figures/{DataFrame}"
                    if not os.path.exists(figlocation):
                        os.makedirs(figlocation,exist_ok=True)

                    #Change obs/pred column names if necessary
                    EvalDF.rename(columns={'ASO_swe_cm':'y_test','XGBoost_swe_cm':'y_pred'}, inplace=True)

                    #just get swe >1cm
                    EvalDF = EvalDF[EvalDF['y_test']>1]

                    #Parity Plot
                    Performance = SSWEET.parityplot(EvalDF, savfig, watershed, testdate, sim, DataFrame)
                    Performance

                    #Error vs Elevation
                    # SSWEET.Model_Vs(EvalDF,'Elevation_m', 'Error', savfig, watershed, testdate, sim)

                    #make path if it does not exist for evaluation figures
                    figpath = f"./SWEMLv2.0/Evaluation/Figures/{DataFrame}/{sim}/{year}/{output_res}"
                    if not os.path.exists(figpath):
                        os.makedirs(figpath, exist_ok=True)

#                     #spatial
                    cmap = 'viridis' # use seismic for error, viridis or blues for preds/obs
                    var =  'y_pred' #'error'
                    var_short = 'Pred'
                    Title = f'SWEMLv2.0 Model {var_short} {testdate} \n {watershed} River Basin, {output_res}'
                    variant = 'World_Imagery'
                    figname = f"{figpath}/{watershed}_spatial_{var}_{testdate}.png"
                    SSWEET.SpatialAnalysis(EvalDF, markersize, cmap, var,Title, savfig, variant, figname)

#                     #spatial
                    cmap = 'viridis' # use seismic for error, viridis or blues for preds/obs
                    var =  'y_test' #'error'
                    var_short = 'test'
                    Title = f'SWEMLv2.0 Model {var_short} {testdate} \n {watershed} River Basin, {output_res}'
                    variant = 'World_Imagery'
                    figname = f"{figpath}/{watershed}_spatial_{var}_{testdate}.png"
                    SSWEET.SpatialAnalysis(EvalDF, markersize, cmap, var,Title, savfig, variant, figname)


#                     #spatial
                    cmap = 'seismic' # use seismic for error, viridis or blues for preds/obs
                    var =  'error'
                    var_short = 'Error'
                    Title = f'SWEMLv2.0 Model {var_short} {testdate} \n {watershed} River Basin, {output_res}'
                    variant = 'World_Imagery'
                    figname = f"{figpath}/{watershed}_spatial_{var}_{testdate}.png"
                    # SSWEET.SpatialAnalysis(EvalDF, markersize, cmap, var,Title, savfig, variant, figname)

                    # cols =['Volume_Difference', 'Predicted_Volume', 'Observed_Volume']
                    # scaler = 10000
                    # ylab = f'Frozen Water Volume (x{scaler}m3)'
                    # ncol = 3
                    # Title = f'Volumetric Frozen Water Content at Low, Mid, and High Elevation Bands \n {watershed} River Basin {testdate}, {sim}, {output_res}'
                    # figname = f"{figpath}/{watershed}_SWEVol_{var}_{testdate}.png"

                    # Vdf = SSWEET.barplot(EvalDF, cols, scaler, ylab, ncol, Title, savfig, figname)
                    # Vdf['watershed'] = watershed
                    # Vdf['Date'] = testdate

#                     cols =['Mean_SWE_Prediction', 'Mean_SWE_Observation']
#                     scaler = 1
#                     ylab = f'Snow-Water-Equivalent (cm)'
#                     ncol = 3
#                     Title = f'Mean SWE at Low, Mid, and High Elevation Bands \n {watershed} River Basin, {testdate} ,  {sim}, {output_res}'
#                     figname = f"{figpath}/{watershed}_MeanSWE_{var}_{testdate}.png"

                    # SWEdf = SSWEET.barplot(EvalDF, cols, scaler, ylab, ncol, Title, savfig, figname)
                    # SWEdf['watershed'] = watershed
                    # SWEdf['Date'] = testdate


                    cols = ['RMSE', 'KGE', 'PBias', 'R', 'R2', 'MAE']
                    Performance = Performance[cols]
                    Performance['watershed'] = watershed
                    Performance['Date'] = testdate
                    Performance['Resolution'] = output_res
                    Performance['Objective'] = hyperparameters['objective']
                    
                    # Performance['n_estimators'] = bestparams['n_estimators']
                    # Performance['max_depth'] = bestparams['max_depth']
                    # Performance['eta'] = bestparams['eta']
                    # Performance['alldata'] = dataused
                    # Performance['standard_training'] = standard

                    PDF = pd.concat([PDF, Performance])
                    # VDF = pd.concat([VDF, Vdf])
                    # SWEDF = pd.concat([SWEDF, SWEdf])

                    #save Performance Dataframes
                    xgb_model.XGB_Perf_Save(PDF, f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/", f"PDF_{sim}_{output_res}")
                    # xgb_model.XGB_Perf_Save(VDF, f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/", f"VDF_{sim}_{output_res}")
                    # xgb_model.XGB_Perf_Save(SWEDF, f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/", f"SWEDF_{sim}_{output_res}")


In [ ]:
from model_scripts import SSWEET

In [ ]:
#spatial
cmap = 'viridis' # use seismic for error, viridis or blues for preds/obs
var =  'NLDAS' #'error'
var_short = 'Precip'
sim = "Test_WY2021" 
output_res = 1000
year = 2021
figpath = f"./SWEMLv2.0/Evaluation/Figures/{sim}/{year}/{output_res}"
# Title = f'SWEMLv2.0 Model {var_short} {testdate} \n {watershed} River Basin, {output_res}'
# variant = 'OpenStreetMap'
# figname = f"{figpath}/{watershed}_spatial_{var}_{testdate}.png"
SSWEET.SpatialAnalysis(EvalDF, markersize, cmap, var,Title, savfig, variant, figname)

output_res = '1000M_Resolution'
SSWEET.plot_cdf('KGE',output_res, title="SWEMLv2.0 WY Hold Out Cumulative Distribution Function for KGE",
                     xlabel="KGE Value", ylabel="Cumulative Probability")# Notes from model input variable analysis
* Observed SWE skewed to the right, way more low swe values in training than high values
* lots of -9999 values for snotel observations - need to see if bad data processing or unavailable snotel values, this is messing up the site swe anomoaly, mean, and observed values.
* Snotel magnitude is incorrect. find out why snotel data is the wrong magnitude and develop method to only select sites with good values - e.g. remove sites that have incomplete or bad data from the obs dataframe and remove from the gridcell location to snotel dictionary.
* change name of rel column to swe anomoly

In [ ]:
output_res = '1000M_Resolution'
SSWEET.plot_cdf('KGE',output_res, title="SWEMLv2.0 WY Hold Out Cumulative Distribution Function for KGE",
                     xlabel="KGE Value", ylabel="Cumulative Probability")

In [ ]:
DataFrame = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
sim  =  f"Test_WY2021_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
sim  =  f"Test_WY2023_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
sim  =  f"Test_WY2017_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
sim  =  f"Test_WY2023_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2019_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2020_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2021_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2022_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:

DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2023_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
DataFrame = 'Sturm_Seasonality_PrecipVIIRSGeoObsDFs'
sim  =  f"Test_WY2024_1000M_Resolution" 
PDF = pd.read_parquet(f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/PDF_{sim}.parquet")
PDF = PDF.sort_values(by =['KGE', 'RMSE'], ascending =[False, False])
PDF

In [ ]:
PDF[PDF['watershed']=='Taylor']

In [ ]:
#CDF plot
output_res = '1000M_Resolution'
SSWEET.plot_cdf('KGE',output_res, title="SWEMLv2.0 WY Hold Out Cumulative Distribution Function for KGE",
                     xlabel="KGE Value", ylabel="Cumulative Probability")

In [ ]:
#save predictions as geotiff to share with Andy Wood
DataFrame = 'NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs'
fSCA_thresh = '10_fSCA_Thresh' 
res= 1000
output_res = f"{res}M_Resolution"
years = [ '2013',
    # '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 
         '2024'
        ]
for year in years:
    print(year)
    directory_path = f"{HOME}/SWEMLv2.0/Predictions/{DataFrame}/{output_res}/{fSCA_thresh}/{year}"
    #file = 'HoldWYsout_1000M_Resolution_USCATB_2014-04-20.parquet'
    Pred_col = 'XGBoost_swe_cm'

    DF_to_GeoTiff.dataframe_to_geotiff(directory_path, Pred_col, Pred_col,
                                 resolution_meters=1000, fill_value=0)

In [ ]:
import rasterio
with rasterio.open(f'{HOME}/SWEMLv2.0/Predictions/NLDASDaymet_Vegetation_Sturm_Seasonality_VIIRSGeoObsDFs/1000M_Resolution/10_fSCA_Thresh/2019/GeoTiffs/HoldWYsout_1000M_Resolution_USCAKW_2019-03-17.tif') as src:
    show(src)

In [ ]:
max(gdf.XGBoost_swe_cm)

In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show


filepath = output_file

# def plot_geotiff(filepath):
#     """
#     Loads and plots a GeoTIFF file to visually inspect its content.

#     Args:
#         filepath (str or Path): The path to the GeoTIFF file.
#     """
filepath = Path(filepath)



# try:
with rasterio.open(filepath) as src:
    # Print some metadata to verify
    print(f"Successfully opened GeoTIFF: {filepath.name}")
    print(f"  CRS: {src.crs}")
    print(f"  Bounds: {src.bounds}")
    print(f"  Resolution: {src.res}")
    print(f"  Width: {src.width}, Height: {src.height}")
    print(f"  Number of bands: {src.count}")
    # Correct ways to get the data type:
    print(src.meta['dtype']) # From the metadata dictionary
    print(src.read(1).dtype) # From the actual NumPy array of the first band
    # print(f"  Data type: {src.dtype}")
    # print(f"  NoData value: {src.nodata}")

    # Read the first band of the raster data
    # If your GeoTIFF has multiple bands, you might need to specify which one
    # or loop through them. For a single-band raster (like precipitation), read(1) is common.
    raster_data = src.read(1)
    print(f"\nSuccessfully read band 1 data. Shape: {raster_data.shape}, Dtype: {raster_data.dtype}")
    print(f"  Min value: {np.nanmin(raster_data):.2f}")
    print(f"  Max value: {np.nanmax(raster_data):.2f}")

    # Create a plot
    fig, ax = plt.subplots(figsize=(10, 10))

    # rasterio.plot.show returns an AxesImage object
    image_plot = show(raster_data, ax=ax, transform=src.transform, cmap='viridis')

    # Add a colorbar
   # if ax.images: # Check if there are any images on the axes
    cbar = fig.colorbar(ax.images[0], ax=ax) # Use the first image added to the axes
    cbar.set_label('SWE (cm)')
                

    # Set title and labels
    ax.set_title(f"GeoTIFF Plot: {filepath.name}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.show()

# except rasterio.errors.RasterioIOError as e:
#     print(f"Error reading GeoTIFF file: {e}. The file might be corrupted or not a valid GeoTIFF.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")



In [ ]:
image_plot

In [ ]:

    import geopandas as gpd
    from shapely.geometry import Point

    # Create a dummy GeoDataFrame
    dummy_gdf_data = {
        'value': [10, 20, 30, 40],
        'geometry': [Point(0.1, 0.1), Point(0.2, 0.2), Point(0.3, 0.3), Point(0.4, 0.4)]
    }
    dummy_gdf = gpd.GeoDataFrame(dummy_gdf_data, crs="EPSG:4326")

    output_dir = Path("temp_geotiff_plot_test")
    output_dir.mkdir(exist_ok=True)
    test_geotiff_path = output_dir / "test_output.tif"

    # Call the geodataframe_to_geotiff function (assuming it's available in your environment)
    # You would typically have this function defined in your script or imported.
    # For this example, we'll use a placeholder if the actual function isn't run.
    # If you've run the Canvas code, you can call it directly:
    # from your_module import geodataframe_to_geotiff # if saved to a module
    # created_file = geodataframe_to_geotiff(dummy_gdf, test_geotiff_path, 'value', resolution_meters=100)

    # --- Placeholder for demonstration if geodataframe_to_geotiff is not run ---
    # If the above function call is commented out or not available,
    # this will create a simple raster directly for plotting demonstration.
    if not test_geotiff_path.exists():
        print("Creating a simple dummy GeoTIFF for plotting demonstration...")
        from rasterio.transform import Affine
        transform = Affine(0.1, 0, 0, 0, -0.1, 10) # Simple transform
        with rasterio.open(
            test_geotiff_path,
            'w',
            driver='GTiff',
            height=10,
            width=10,
            count=1,
            dtype=np.float32,
            crs='EPSG:4326',
            transform=transform,
            nodata=-9999
        ) as dst:
            dummy_data = np.random.rand(10, 10) * 100
            dst.write(dummy_data, 1)
        print("Dummy GeoTIFF created for plotting test.")
    # --- End Placeholder ---

    # Now, plot the created GeoTIFF
    plot_geotiff(test_geotiff_path)

# finally:
    # Clean up dummy files and directory
    if 'output_dir' in locals() and output_dir.exists():
        print("\nCleaning up dummy files and directory...")
        for f in output_dir.iterdir():
            f.unlink()
        output_dir.rmdir()
        print("Cleanup complete.")

In [ ]:

#     # Create a dummy DataFrame (similar to what you'd get from Earth Engine sampling)
#     print("Creating dummy DataFrame for demonstration...")
#     data = {
#         'longitude': np.linspace(-120.0, -119.0, 10),
#         'latitude': np.linspace(37.7, 38.2, 10),
#         'precipitation_mm': np.random.rand(10) * 100 # Random precipitation values
#     }
#     # Create a grid-like DataFrame for better rasterization example
#     lons = np.arange(-120.0, -119.0, 0.125) # NLDAS-like resolution
#     lats = np.arange(37.7, 38.2, 0.125)
    
#     # Create a meshgrid for all combinations
#     lon_grid, lat_grid = np.meshgrid(lons, lats)
    
#     # Flatten grids and create random values
#     dummy_df_data = {
#         'longitude': lon_grid.flatten(),
#         'latitude': lat_grid.flatten(),
#         'precipitation_mm': np.random.rand(len(lon_grid.flatten())) * 100
#     }
#     dummy_df = pd.DataFrame(dummy_df_data)

#     # Add some missing data points to test nodata_value
#     dummy_df.loc[::5, 'precipitation_mm'] = np.nan # Introduce some NaNs

#     print("Dummy DataFrame Head:")
#     print(dummy_df.head())
#     print(f"Dummy DataFrame Shape: {dummy_df.shape}")

    # # Define output path
    # output_dir = Path("output_rasters")
    # output_dir.mkdir(exist_ok=True)
    # output_file = output_dir / "monthly_precip_raster.tif"

    # Convert DataFrame to GeoTIFF
    print("\nAttempting to convert DataFrame to GeoTIFF...")
    created_file = dataframe_to_geotiff(
        dummy_df,
        output_file,
        value_column='precipitation_mm',
        resolution=0.125, # Explicitly set NLDAS resolution
        nodata_value=-9999 # A common nodata value
    )

    if created_file:
        print(f"\nGeoTIFF created successfully at: {created_file}")
        # Optional: Verify by opening with rasterio
        try:
            with rasterio.open(created_file) as src:
                print(f"Verified GeoTIFF: {src.meta}")
                print(f"Bounds: {src.bounds}")
                print(f"Nodata value: {src.nodata}")
                # Read a small part of the data
                data_read = src.read(1)
                print(f"Data array shape: {data_read.shape}")
                print(f"Data array sample (top-left): {data_read[0, 0:5]}")
        except Exception as e:
            print(f"Error verifying GeoTIFF: {e}")

    # Clean up dummy directory and files
    print("\nCleaning up dummy files and directory...")
    if output_dir.exists():
        for f in output_dir.iterdir():
            f.unlink()
        output_dir.rmdir()
    print("Cleanup complete.")